# Simple ORT UI

This is only a proof of concept using basic widgets provided by `ipywidget`. 

In [ ]:
import re
import os
import shutil
import tempfile
import subprocess
from os.path import expanduser, exists, join
from functools import partial

In [ ]:
from IPython.display import display
from ipywidgets import HTML, Tab, HBox, VBox, Button, ButtonStyle, Layout, Textarea, Text, Output, Dropdown, Checkbox
from halo import HaloNotebook

In [ ]:
from ortung import clone_repo, parse_repo_url

## Callbacks

In [ ]:
output = Output()

In [ ]:
@output.capture()
def clone_repo_clicked(button,
               source=None, url=None, branch=None, dest_dir=None,
               overwrite=True, verbose=False):
    """Clone a repo into some destination folder.
    """
    dest_dir = expanduser(dest_dir or ".")
    if not os.path.exists(dest_dir):
        with output:
            print(f"{dest_dir} does not exist.")
        return

    source = source.value
    url = url.value
    branch = branch.value

    # determine repo full URL, owner and name 
    if re.match("http[s]\://.*\.git", url):
        full_url = url
        owner, repo = full_url[:-4].split("/")[-2:]
    elif source == "GitHub":
        full_url = f"https://github.com/"
        if "/" in url:
            if url.endswith(".git"):
                full_url += url
            else:
                full_url += f"{url}.git"
        owner, repo = full_url[:-4].split("/")[-2:]

    if verbose:
        with output:
            print(f"{url} {dest_dir} {full_url} {owner} {repo}")    
    with HaloNotebook(text=f"Cloning {full_url}", spinner='dots'):
        out = clone_repo(full_url, branch=branch, dest_dir=dest_dir)

In [ ]:
@output.capture()
def analyze_clicked(button,
                    url=None,
                    source_dir=None,
                    verbose=False):
    """Analyze a repo.
    """
    source_dir = expanduser(source_dir or ".")
    if not os.path.exists(source_dir):
        with output:
            print(f"{source_dir} does not exist.")
        return

    url = url.value
    name = parse_repo_url(url)["name"]
    cmd = ["ort", 
           "--debug", 
           "--stacktrace", 
           "analyze",
           "-f", "JSON", 
           "-i", f"{source_dir}/{name}", 
           "-o", f"{source_dir}/{name}-ort/analyzer", 
           "--allow-dynamic-versions"]
    if False: # verbose:
        with output:
            print(cmd)
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    with HaloNotebook(text=f"Analysing: {' '.join(cmd)}", spinner='dots'):
        while proc.poll() is None:
            line = proc.stdout.readline().decode("utf-8")
            if False: # verbose:
                with output:
                    print(line.strip())

## UI

In [ ]:
# defaults
repo = "https://github.com/jshttp/mime-types.git"
branch = "2.1.18"

In [ ]:
# UI
layout = Layout(width="100%")
layout1 = Layout(width="75%")
title = HTML("<strong>ORT Scan for Repository</strong>")
source_dd = Dropdown(description="Source", options=["", "GitHub", "GitLab", "BitBucket"])
url_tx = Text(description="Name/URL", value=repo, layout=layout1)
branch_tx = Text(description="Branch/tag/commit", value=branch, layout=layout1)
style = ButtonStyle(button_color='#48dad0')
overwrite_cb = Checkbox(description="Overwrite", value=True)
verbose_cb = Checkbox(description="Verbose", value=True)

source = source_dd.value
url = url_tx.value
branch = branch_tx.value
overwrite = overwrite_cb.value
verbose = verbose_cb.value

start_btn = Button(description="Clone", style=style)
callback = partial(clone_repo_clicked, 
                   source=source_dd, url=url_tx, branch=branch_tx, dest_dir="tmp",
                   overwrite=overwrite, 
                   verbose=verbose)
start_btn.on_click(callback)

analyze_btn = Button(description="Analyze", style=style)
callback2 = partial(analyze_clicked,
                    url=url_tx, source_dir="tmp",
                    verbose=verbose)
analyze_btn.on_click(callback2)

ui = VBox(children=[
    title, 
    source_dd, 
    url_tx, 
    branch_tx, 
    HBox(children=[start_btn, analyze_btn]), 
    HBox(children=[
        overwrite_cb,
        verbose_cb
    ]), 
], layout=layout)

In [ ]:
ui

In [ ]:
output